# NBA DATA

Read the data from postgres database and prepare the data for training

In [52]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.endpoints import TeamGameLogs ,playergamelogs
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import URL
import pg8000
import yaml
from types import SimpleNamespace

In [53]:
# Reading the config YAML file
with open('./config.yaml', 'r') as file:
    config_dict = yaml.safe_load(file)
cfg = SimpleNamespace(**config_dict)

In [2]:

# Database connection parameters
DB_HOST = "localhost"  # Use container IP if needed
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "nba_data"
DB_USER = "mtzimas"
DB_PASSWORD = "what2beradera"

# Create a connection to PostgreSQL using SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


Teams info

In [3]:
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)
df_teams.head(3)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970


Active players info

In [4]:
active_players = players.get_active_players()
df_players = pd.DataFrame(active_players)
df_players.head(3)

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True


Season 2023-24 stats per game

In [62]:
# Select a table to load as a DataFrame
table_name = "season23_games"
# Read the table into a Pandas DataFrame
games_df = pd.read_sql(f"SELECT * FROM {table_name}", engine)
games_df.head(5)

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,2023-24,1610612757,POR,Portland Trail Blazers,0022301200,2024-04-14T00:00:00,POR @ SAC,L,48.0,31,...,2359,2077,205,2080,1459,1208,1020,2449,2437,1
1,2023-24,1610612745,HOU,Houston Rockets,0022301199,2024-04-14T00:00:00,HOU @ LAC,W,48.0,50,...,423,2077,1161,267,2012,143,2081,1044,547,1
2,2023-24,1610612753,ORL,Orlando Magic,0022301191,2024-04-14T00:00:00,ORL vs. MIL,W,48.0,42,...,1031,1228,205,1003,710,968,1254,1243,127,1
3,2023-24,1610612751,BKN,Brooklyn Nets,0022301192,2024-04-14T00:00:00,BKN @ PHI,L,48.0,32,...,2294,991,839,690,1459,250,1725,2430,2243,1
4,2023-24,1610612738,BOS,Boston Celtics,0022301186,2024-04-14T00:00:00,BOS vs. WAS,W,48.0,51,...,678,1920,351,4,710,143,1725,186,598,1


Season 23-24 player stats per game

In [6]:
table_name = "player_game_logs_23"
players_stats_df = pd.read_sql(f"SELECT * FROM {table_name}", engine)
players_stats_df['GAME_DATE'] = pd.to_datetime(players_stats_df['GAME_DATE'])
players_stats_df.head(5)

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,MIN_SEC
0,2023-24,1629670,Jordan Nwora,Jordan,1610612761,TOR,Toronto Raptors,0022301189,2024-04-14,TOR @ MIA,...,6163,17532,7525,16567,8894,1,138,7405,1,20:31
1,2023-24,1630581,Josh Giddey,Josh,1610612760,OKC,Oklahoma City Thunder,0022301196,2024-04-14,OKC vs. DAL,...,13042,11166,13454,1570,10260,2248,138,10143,1,19:31
2,2023-24,1631093,Jaden Ivey,Jaden,1610612765,DET,Detroit Pistons,0022301197,2024-04-14,DET @ SAS,...,1,6969,8427,21324,15911,2248,138,13968,1,27:04
3,2023-24,1630541,Moses Moody,Moses,1610612744,GSW,Golden State Warriors,0022301198,2024-04-14,GSW vs. UTA,...,1,6969,12379,3565,13948,2248,138,13968,1,16:57
4,2023-24,1630169,Tyrese Haliburton,Tyrese,1610612754,IND,Indiana Pacers,0022301188,2024-04-14,IND vs. ATL,...,18879,6969,9297,350,2337,1,138,4374,1,25:25


In [7]:
def home_away_id(games_df, game_id):
    """"
    Returns the home and away team IDs based on the game matchup string.

    Parameters:
    - games_df (pd.DataFrame): DataFrame containing game details with a 'MATCHUP' column.
    - game_id (int or str): The unique GAME_ID to find the matchup.

    Returns:
    - tuple: (home_team_id, away_team_id)
    """
    
    mat = games_df[games_df['GAME_ID']==game_id]['MATCHUP'].iloc[0]
    if '@' in mat:
        teams_abb = mat.split(' @ ')
        home_team_id = df_teams[df_teams['abbreviation']==teams_abb[1]]['id'].item()
        away_team_id = df_teams[df_teams['abbreviation']==teams_abb[0]]['id'].item()
    else : 
        teams_abb = mat.split('vs.')
        home_team_id = df_teams[df_teams['abbreviation']==teams_abb[0]]['id'].item()
        away_team_id = df_teams[df_teams['abbreviation']==teams_abb[1]]['id'].item()

    return home_team_id, away_team_id


In [81]:
def to_avg_string(columns):
    """
    Transforms a list of columns to a string for sql query automation.
    """
    query_string = ", ".join([f'AVG("{col}") AS "AVG_{col}"' for col in columns])
    return query_string

def game_roster(players_log_df, games_df, game_id):
    """
    Retrieves the home and away team rosters for a given game.
    In addition as we dont have an endpoint of injuries per date we use the stats info to know the roster for a given date.
    This function also retrives 

    Parameters:
    - players_log_df (pd.DataFrame): A DataFrame containing player statistics with "GAME_ID" and "TEAM_ID".
    - games_df (pd.DataFrame): A DataFrame containing game information, including matchups.
    - game_id (int or str): The unique identifier of the game.

    Returns:
    - tuple: (home_players_stats, away_players_stats)
      - home_players_stats (pd.DataFrame): Player statistics of the home team for game_id
      - away_players_stats (pd.DataFrame): Player statistics of the away team for game_id.
      - home_players_season_stats (pd.DataFrame): Season player statistics of the home team before game_id
      - away_players_season_stats (pd.DataFrame): Season player statistics of the away team before game_id. 
    """

    home_id, away_id = home_away_id(games_df,game_id)
    game_players_stats = players_log_df[players_log_df["GAME_ID"]==game_id]
    
    home_players_stats = game_players_stats[game_players_stats["TEAM_ID"]==home_id]
    away_players_stats = game_players_stats[game_players_stats["TEAM_ID"]==away_id]

    # date of the given game id
    gameid_date = games_df[games_df['GAME_ID']==game_id]['GAME_DATE'].iloc[0]

    player_stat_query = f"SELECT \"PLAYER_ID\",\"TEAM_ID\", {to_avg_string(cfg.keep_player_stats)}\
                        FROM player_game_logs_23\
                        WHERE TO_DATE(\"GAME_DATE\", \'YYYY-MM-DD\') < \'{gameid_date}\'\
                        AND \"TEAM_ID\" IN (\'{home_id}\', \'{away_id}\')\
                        GROUP BY \"PLAYER_ID\", \"TEAM_ID\";"

    players_season_stats = pd.read_sql(player_stat_query, engine)

    home_players_season_stats = players_season_stats[players_season_stats['PLAYER_ID'].isin(list(home_players_stats['PLAYER_ID']))]
    away_players_season_stats = players_season_stats[players_season_stats['PLAYER_ID'].isin(list(away_players_stats['PLAYER_ID']))]

    return home_players_stats, away_players_stats, home_players_season_stats, away_players_season_stats

game_roster(players_log_df=players_stats_df, games_df=games_df, game_id="0022301196")[3].head(20)

,PLAYER_ID,TEAM_ID,AVG_MIN,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,...,AVG_REB,AVG_AST,AVG_TOV,AVG_STL,AVG_BLK,AVG_BLKA,AVG_PF,AVG_PFD,AVG_PTS,AVG_PLUS_MINUS
8,1630182,1610612742,26.311280,3.178571,6.535714,0.450714,1.285714,3.267857,0.330696,0.660714,...,3.232143,2.267857,1.053571,0.839286,0.160714,0.535714,1.910714,1.303571,8.303571,-1.000000
10,1630639,1610612742,7.008943,1.195122,2.634146,0.380927,0.317073,1.097561,0.120220,0.317073,...,1.097561,0.439024,0.317073,0.243902,0.073171,0.195122,0.512195,0.292683,3.024390,0.878049
24,1630702,1610612742,13.338449,2.652778,6.458333,0.377875,1.111111,3.055556,0.267306,0.819444,...,1.763889,1.472222,0.805556,0.277778,0.055556,0.347222,1.041667,1.069444,7.236111,-1.388889
28,1641765,1610612742,7.664402,0.897436,2.307692,0.243462,0.282051,1.000000,0.100436,0.641026,...,1.820513,0.538462,0.153846,0.179487,0.102564,0.128205,0.461538,0.589744,2.717949,-1.384615
34,1641788,1610612742,6.833333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000
35,203939,1610612742,13.053548,1.145161,1.677419,0.446274,0.016129,0.032258,0.016129,1.016129,...,3.322581,1.338710,0.500000,0.403226,0.306452,0.193548,1.774194,1.677419,3.322581,0.548387
37,202693,1610612742,7.708200,0.840000,2.440000,0.331840,0.520000,1.520000,0.226000,0.120000,...,1.360000,0.520000,0.400000,0.200000,0.080000,0.080000,0.480000,0.280000,2.320000,-1.640000
39,1630314,1610612742,5.192917,0.750000,2.062500,0.317750,0.125000,0.562500,0.093750,0.375000,...,0.500000,0.875000,0.250000,0.062500,0.062500,0.250000,0.187500,0.687500,2.000000,-1.750000
42,203501,1610612742,26.931560,5.012821,12.397436,0.386154,2.692308,7.576923,0.334000,1.846154,...,3.243590,1.743590,0.884615,0.512821,0.076923,0.423077,1.333333,2.025641,14.564103,0.576923


In [46]:
game_roster(players_log_df=players_stats_df, games_df=games_df, game_id="0022301196")[1]['PTS'].sum()

86

In [64]:
games_df[games_df['GAME_ID']=="0022301196"]['GAME_DATE'].iloc[0]


'2024-04-14T00:00:00'